# <b> 1. LSA(Latent Semantic Analysis)</b> 

In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import TreebankWordTokenizer

In [2]:
df_2024 = pd.read_csv('../드라마영화예능_전처리데이터/24영화_전처리.csv')
df_2023 = pd.read_csv('../드라마영화예능_전처리데이터/23영화_전처리.csv')
df_2022 = pd.read_csv('../드라마영화예능_전처리데이터/22영화_전처리.csv')

df = pd.concat([df_2024, df_2023, df_2022], axis=0).reset_index(drop=True)

df.head()

,postdate,body,description,title,preprocessed_body,preprocessed_description,preprocessed_title
0,20231219,성수동 전시 디즈니100주년 기념 팝업 House of Wish 디즈니 2024년 ...,성수동 전시 디즈니100주년 기념 팝업 House of Wish 디즈니 b2024년...,성수동 전시 디즈니100주년 기념 팝업 House of Wish 디즈니 2024년 ...,성수동 전시 디즈니 주년 기념 팝업 디즈니 위시 블로거 새틴 인플루언서새틴 일요일 ...,성수동 전시 디즈니 주년 기념 팝업 디즈니 위 시 블로거 새틴 토 수 운영 디즈니 ...,성수동 전시 디즈니 주년 기념 팝업 디즈니 위시
1,20240202,안녕하세요 죠로링입니다 저는 19년도부터 넷플릭스를 구독하여 지금까지 친구들과 아이...,이유 b2024년b 개봉 예정작 저는 계정을 공유하고 있지만 넷플릭스를 잘 보지 않...,넷플릭스 계정 공유 방법 모바일 단속 시행 2024년 넷플 개봉작 추천,안녕하세요 죠로링 년도 넷플릭스 구독 친구 아이디 공유 사용 작년 넷플릭스 금액 인...,이유 계정 공유 넷플릭스 데 넷플릭스 코리아 인스타 주,넷플릭스 계정 공유 방법 모바일 단속 시행 넷플
2,20240102,1월 개봉예정영화 2024년 최신 개봉작 한국 해외 외국 영화 추천 벌써 2일이 시...,1월 개봉예정영화 b2024년b 최신 b개봉작b 한국 해외 외국 영화 추천 벌써 2...,1월 개봉예정영화 2024년 최신 개봉작 한국 해외 외국 영화 추천,해외 외국 도 느낌 느낌 데 작년 도 내일 디즈니 위시 글 포스팅 번 김 체크 길 ...,해외 외국 도 느낌 느낌 데 작년 도,해외 외국
3,20240205,오늘은 2024년 2월 메가박스 돌비시네마 개봉작에 관한 글을 적어볼까 합니다 2월...,b2024년b 2월 7일 돌비시네마 상영작은 아가일 b2024년b 2월 7일 b개봉...,2024년 2월 메가박스 돌비시네마 개봉작 소개,은 메가박스 돌비 시네마 적 돌비 은 패스 애매 적 아이맥스 돌비 시네마 은 초 말...,돌비 시네마 상영작 아가 아가 은 돌비 시네마 포맷 가일 돌비 비전 돌비 애트모스 ...,메가박스 돌비 시네마
4,20240501,이 포스팅은 2024년 5월 개봉 예정 영화 전체를 소개하는 소개글이 아닌 저 스스...,b2024년b 5월 개봉 관람 예정 영화 리스트 흥행 괴물 형사의 독주를 막을 가정...,2024년 5월 개봉 관람 예정 영화 리스트 흥행 괴물 형사의 독주를 막을 가정의 ...,포스팅 전체 글 고민 목표 자체 고민 인하 개인 취향 선정 만 괴물 형사 독주 가정...,괴물 형사 독주 가정 달 승자 길 비수기 성수기 진입 가정 달 니,괴물 형사 독주 가정 달 승자


In [3]:
# 결측치 제거
df = df.dropna(subset=['preprocessed_body'])
df.shape

(1902, 7)

In [4]:
# 상위 (max_features)개의 단어에 대해서만 vectorize  -> 각각의 문서들을 벡터 형태로 만들기 위해서 사용
# max_df=0.5: 문서의 절반 이상에서 나타나는 단어를 무시 -> 자주 등장하지만 정보량이 적은 단어 
vectorizer = TfidfVectorizer(max_features=300, max_df=0.5, smooth_idf=True) 
X = vectorizer.fit_transform(df['preprocessed_body'])

In [5]:
print(X.shape)

(1902, 300)


In [6]:
# X는 sparse matrix. 
# 효율적인 저장 방식인 CSR(Compressed Sparse Row) matrix 형식으로 저장되어 있음
X

<1902x300 sparse matrix of type '<class 'numpy.float64'>'
	with 63446 stored elements in Compressed Sparse Row format>

- <span style = 'font-size:1.1em;line-height:1.5em'>TF-IDF의 각 차원의 의미</span>

In [7]:
terms = vectorizer.get_feature_names_out()
print(terms) # 벡터의 각 차원이 어떤 단어를 의미하는지 
print(len(terms)) # 총 벡터의 차원은 300차원 

['가능' '가디언즈' '가운데' '가지' '각본' '감동' '감상' '감정' '개인' '갤럭시' '게임' '결과' '결심' '경우'
 '경험' '고양이' '공간' '공조' '공포' '과거' '과정' '관련' '관심' '괴물' '국가' '국제' '귀멸의' '규모'
 '극장가' '극장판' '금주' '기간' '기억' '기자' '나라' '나름' '남자' '남편' '네이버' '넷플릭스' '년대'
 '노래' '노량' '녹음' '느낌' '능력' '다니엘' '다양' '다큐멘터리' '닥터' '달러' '당시' '대결' '대사' '대표'
 '덩크' '데드' '데뷔' '도시' '독립' '돌비' '동시' '동원' '등급' '디즈니' '러닝' '러브' '로맨스' '로스트'
 '로튼' '롯데시네마' '마녀' '마동석' '마블' '마을' '마음' '마이클' '마지막' '매력' '매버릭' '매출' '멀티'
 '메가박스' '모험' '목소리' '무대' '무비' '문단속' '문제' '뮤지컬' '미국' '미션' '미스터리' '바다' '반응'
 '배경' '배급' '배트맨' '버스' '범죄' '범죄도시' '부문' '부분' '북미' '분기점' '분위기' '불가' '브라더스'
 '블랙' '블록버스터' '비교' '비밀' '사건' '사실' '사이' '사진' '사회' '살인' '상영관' '상태' '샬라메'
 '서비스' '서울' '선사' '선언' '선택' '성공' '세상' '소녀' '소년' '소니' '소설' '소식' '소재' '속편'
 '손익' '수요일' '수입' '수준' '순간' '슈퍼' '스릴러' '스즈메의' '스크린' '스타' '스토리' '스튜디오'
 '스트레인지' '스파이' '스파이더맨' '시네마' '시대' '시민' '시장' '시절' '시즌' '시청' '실패' '실화' '아들'
 '아래' '아바타' '아이맥스' '아카데미' '애니메이션' '얼굴' '엄마' '여름' '여성' '여자' '역사' '역할' '연휴'
 '영상' '영화관' '예상' '예술' '오리지널' '오브' '오펜하이머' '오프닝' '와이드'

- <span style = 'font-size:1.1em;line-height:1.5em'>TF-IDF matrix에 Truncated SVD를 적용하여 LSA 수행</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>n_components = 토픽 수</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>일반적으로 TF-IDF 특징 수의 5%에서 20% 사이의 값을 토픽 수로 설정

In [8]:
model = TruncatedSVD(n_components=30, n_iter=100, random_state=42) 
model.fit(X)

TruncatedSVD(n_components=30, n_iter=100, random_state=42)

$$X=U \Sigma V^{T} \approx U_{trunc} \Sigma_{trunc} V_{trunc}^{T}$$ 
- <span style = 'font-size:1.2em;line-height:1.5em'>여기서 <code>result=model.transform(X)</code>: $U_{trunc} \Sigma_{trunc}$</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>1902개의 문서를 300차원의 벡터로 (즉, 300개 단어의 조합으로) 표현</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>그러나, SVD를 활용하여 1902개의 문서를 30차원의 벡터로 (즉, 30개 주제의 조합으로) 표현 </span>

In [9]:
result = model.transform(X)
print(result.shape)

(1902, 30)


In [10]:
result.shape

(1902, 30)

In [11]:
# 30개의 각 topic이 어떻게 300개의 단어 조합으로 되어있는지
print(model.components_.shape)
print(model.components_)

(30, 300)
[[ 0.06453034  0.03246838  0.03924108 ...  0.05384647  0.04695393
   0.04542208]
 [ 0.0102506   0.03613421 -0.00138581 ...  0.01181501 -0.01921107
   0.02794572]
 [-0.00715427  0.01969584  0.00173936 ... -0.01883875 -0.02572491
  -0.00867244]
 ...
 [-0.02420141  0.03381959 -0.01558865 ...  0.03124103  0.03123426
   0.02683243]
 [-0.01025321  0.09946739 -0.01292832 ... -0.10543208 -0.00217612
   0.03236418]
 [-0.02070121 -0.02937038  0.0157971  ...  0.06825178  0.02208865
  -0.01660035]]


In [12]:
terms = vectorizer.get_feature_names_out()
print(terms) # 벡터의 각 차원이 어떤 단어를 의미하는지 (Each dimension is matched with which word)
print(len(terms)) # 총 벡터의 차원은 300차원 (Dimension: 300)

['가능' '가디언즈' '가운데' '가지' '각본' '감동' '감상' '감정' '개인' '갤럭시' '게임' '결과' '결심' '경우'
 '경험' '고양이' '공간' '공조' '공포' '과거' '과정' '관련' '관심' '괴물' '국가' '국제' '귀멸의' '규모'
 '극장가' '극장판' '금주' '기간' '기억' '기자' '나라' '나름' '남자' '남편' '네이버' '넷플릭스' '년대'
 '노래' '노량' '녹음' '느낌' '능력' '다니엘' '다양' '다큐멘터리' '닥터' '달러' '당시' '대결' '대사' '대표'
 '덩크' '데드' '데뷔' '도시' '독립' '돌비' '동시' '동원' '등급' '디즈니' '러닝' '러브' '로맨스' '로스트'
 '로튼' '롯데시네마' '마녀' '마동석' '마블' '마을' '마음' '마이클' '마지막' '매력' '매버릭' '매출' '멀티'
 '메가박스' '모험' '목소리' '무대' '무비' '문단속' '문제' '뮤지컬' '미국' '미션' '미스터리' '바다' '반응'
 '배경' '배급' '배트맨' '버스' '범죄' '범죄도시' '부문' '부분' '북미' '분기점' '분위기' '불가' '브라더스'
 '블랙' '블록버스터' '비교' '비밀' '사건' '사실' '사이' '사진' '사회' '살인' '상영관' '상태' '샬라메'
 '서비스' '서울' '선사' '선언' '선택' '성공' '세상' '소녀' '소년' '소니' '소설' '소식' '소재' '속편'
 '손익' '수요일' '수입' '수준' '순간' '슈퍼' '스릴러' '스즈메의' '스크린' '스타' '스토리' '스튜디오'
 '스트레인지' '스파이' '스파이더맨' '시네마' '시대' '시민' '시장' '시절' '시즌' '시청' '실패' '실화' '아들'
 '아래' '아바타' '아이맥스' '아카데미' '애니메이션' '얼굴' '엄마' '여름' '여성' '여자' '역사' '역할' '연휴'
 '영상' '영화관' '예상' '예술' '오리지널' '오브' '오펜하이머' '오프닝' '와이드'

In [13]:
model.components_[0].shape

(300,)

- <span style = 'font-size:1.2em;line-height:1.5em'>각 topic에 대해서 상위 5개의 큰 값을 갖는 단어를 출력하는 함수</span>

In [14]:
def get_keyword_by_topic(components, feature_name, n=5):
    for idx, topic in enumerate(components):
        sorted_keyword_idx = np.argsort(components[idx])[-1:-n-1:-1]
        sorted_keyword_result = [(feature_name[x], topic[x].round(4)) for x in sorted_keyword_idx]
        print(f'Topic {idx}: {sorted_keyword_result}')

In [15]:
get_keyword_by_topic(model.components_, terms)

Topic 0: [('넷플릭스', 0.1593), ('애니메이션', 0.156), ('범죄도시', 0.1393), ('달러', 0.137), ('공포', 0.1198)]
Topic 1: [('달러', 0.536), ('북미', 0.3642), ('월드', 0.222), ('해외', 0.2027), ('와이드', 0.192)]
Topic 2: [('범죄도시', 0.701), ('마동석', 0.2903), ('범죄', 0.2455), ('파묘', 0.1696), ('도시', 0.1624)]
Topic 3: [('북미', 0.3921), ('파묘', 0.2382), ('해외', 0.2193), ('웡카', 0.1923), ('넷플릭스', 0.1876)]
Topic 4: [('넷플릭스', 0.7354), ('범죄도시', 0.2497), ('달러', 0.2048), ('마동석', 0.1336), ('범죄', 0.1193)]
Topic 5: [('기자', 0.3931), ('파묘', 0.3789), ('웡카', 0.231), ('독립', 0.2145), ('녹음', 0.2049)]
Topic 6: [('프랑스', 0.4978), ('경우', 0.3993), ('달러', 0.2753), ('범죄도시', 0.0868), ('북미', 0.0726)]
Topic 7: [('프랑스', 0.5367), ('경우', 0.4571), ('마블', 0.2491), ('넷플릭스', 0.2324), ('파묘', 0.2061)]
Topic 8: [('문단속', 0.2842), ('스즈메의', 0.2838), ('애니메이션', 0.2817), ('디즈니', 0.2775), ('덩크', 0.1665)]
Topic 9: [('극장판', 0.3963), ('추석', 0.2021), ('애니메이션', 0.2018), ('넷플릭스', 0.1841), ('극장가', 0.1459)]
Topic 10: [('외계인', 0.4033), ('웡카', 0.3654), ('디즈니', 0.1705), ('시민', 0

# <b> 2. LDA(Latent Dirichlet Allocation)</b> 

In [16]:
# preprocessed_body에 있는 단어를 리스트에 넣은 칼럼 생성
df['word_list'] = df['preprocessed_body'].apply(lambda x: x.split())

In [17]:
df['word_list'].value_counts()

word_list
[본문, 수]                                                                                                                                                                                                                                                                                                                                                                                                                                                                       36
[삶, 거, 거, 거, 마음가짐, 준비, 보, 수, 시, 주, 시, 분, 감사, 외국, 편, 심사숙고, 끝, 선택, 외국, 중, 스포일러, 주의, 위, 말, 소녀, 말, 지, 수, 수, 수, 앞, 말, 위, 가능, 존재, 연인, 번, 지, 위, 낙엽, 낙엽, 로맨스, 입맞춤, 연애, 아키, 카우리스마키, 은, 낭만, 거, 연애, 수, 위, 애프터, 썰, 하늘, 아래, 이제, 시절, 키, 은, 햇살, 아래, 그림자, 그림자, 은, 추억, 노력, 위, 이니셰린의, 밴시, 다정, 이유, 다정, 의미, 수록, 파우릭이, 콜름이, 듯, 다정, 거, 세월, 면, 수록, 연, 준비, 다정, 눈, 다정, 의미, 질문, 다정, 이유, 답, 중, 말, 위, 왕국, 왕국, 파편, 여기저기, ...]                                                                               2
[안녕하세요, 니, 마동석, 범죄도시, 시, 어제, 중, 오랫만, 시리즈물, 성공, 범죄도시, 시, 성적표,

In [18]:
lst = df['word_list'].apply(lambda words: [word for word in words if len(word) == 1])
lst
# word_list에서 한 글자인 단어 확인 -> 무의미하다고 판단

0       [길, 수, 번, 위, 시, 뿐, 라, 리, 꿈, 거, 시, 토, 수, 롭, 명, ...
1       [기, 플, 도, 게, 살, 번, 번, 면, 고, 몰, 니, 번, 면, 명, 데, ...
2       [도, 데, 도, 글, 번, 김, 길, 글, 위, 시, 글, 위, 시, 평, 데, ...
3       [은, 적, 은, 적, 은, 초, 말, 말, 듄, 거, 은, 전, 지, 과, 거, ...
4       [글, 만, 달, 길, 달, 니, 시, 중, 데, 되, 날, 도, 수, 거, 앞, ...
                              ...                        
1900    [은, 고, 분, 백, 지, 니, 님, 키, 형, 인, 후, 인, 니, 후, 방, ...
1901                 [콘, 톰, 탑, 걸, 앤, 탑, 건, 존, 윅, 킹, 윌, 투]
1902    [데, 도, 데, 거, 듯, 인, 조, 데, 말, 핫, 행, 위, 줄, 랩, 고, ...
1903    [차, 피, 물, 수, 피, 위, 위, 피, 디, 주, 차, 수, 위, 주, 차, ...
1904    [은, 거, 거, 주, 시, 개, 분, 율, 위, 개, 시, 명, 번, 은, 뒤, ...
Name: word_list, Length: 1902, dtype: object

In [19]:
# word_list에서 한 글자인 단어 삭제
df['word_list'] = df['word_list'].apply(lambda words: [word for word in words if len(word) > 1])
df['word_list'].value_counts()

word_list
[본문]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      36
[마음가짐, 준비, 감사, 외국, 심사숙고, 선택, 외국, 스포일러, 주의, 소녀, 가능, 존재, 연인, 낙엽, 낙엽, 로맨스, 입맞춤, 연애, 아키, 카우리스마키, 낭만, 연애, 애프터, 하늘, 아래, 이제, 시절, 햇살, 아래, 그림자, 그림자, 추억, 노력, 이니셰린의, 밴시, 다정, 이유, 다정, 의미, 수록, 파우릭이, 콜름이, 다정, 세월, 수록, 준비, 다정, 다정, 의미, 질문, 다정, 이유, 왕국, 왕국, 파편, 여기저기, 지금, 미야자키, 하야오, 파편, 만큼, 미야자키, 하야오, 시절, 지브리, 스튜디오, 플라워, 킬링, 사악, 멜로, 멜로, 악의, 마틴, 스콜세지, 사악, 멜로, 악의, 취하, 의심, 의심, 반복, 타르, 지목, 과정, 세상, 사냥, 세상, 지목, 평가, 예술가, 혐의, 용의자, 소문, 인간, 선택, 라면, 케이트, 블란쳇, 캐롤, 타르, ...]                                               2
[안녕하

In [20]:
from gensim import corpora

word_dict = corpora.Dictionary(df['word_list'])

In [21]:
corpus = [word_dict.doc2bow(text) for text in df['word_list']]

In [22]:
len(corpus)

1902

In [23]:
corpus[2]  ## 2번째 문서에 대해 각각의 단어들이 몇 번을 나왔는지.. 
## -> (1, 1): 1번 단어가 1번 나오고... 
## -> (4, 3): 4번 단어가 3번 나오고...

[(5, 2),
 (11, 2),
 (28, 2),
 (36, 1),
 (40, 2),
 (61, 1),
 (100, 2),
 (105, 1),
 (109, 1),
 (120, 1),
 (138, 1),
 (185, 1),
 (201, 1),
 (234, 1),
 (251, 1),
 (252, 1),
 (253, 1),
 (254, 1),
 (255, 1),
 (256, 1),
 (257, 1),
 (258, 1),
 (259, 2),
 (260, 1),
 (261, 1),
 (262, 1),
 (263, 2),
 (264, 1),
 (265, 1),
 (266, 1),
 (267, 1),
 (268, 2),
 (269, 1),
 (270, 1),
 (271, 1),
 (272, 1),
 (273, 1),
 (274, 1),
 (275, 1),
 (276, 1),
 (277, 2),
 (278, 1),
 (279, 1),
 (280, 2),
 (281, 1),
 (282, 1),
 (283, 1),
 (284, 3),
 (285, 1),
 (286, 1),
 (287, 1),
 (288, 1),
 (289, 1),
 (290, 1),
 (291, 1),
 (292, 1),
 (293, 1),
 (294, 1),
 (295, 1),
 (296, 3),
 (297, 1),
 (298, 1),
 (299, 1),
 (300, 1),
 (301, 1),
 (302, 1),
 (303, 1),
 (304, 1),
 (305, 1),
 (306, 2),
 (307, 2),
 (308, 1),
 (309, 1),
 (310, 1),
 (311, 1),
 (312, 1),
 (313, 1),
 (314, 2),
 (315, 1),
 (316, 1),
 (317, 1),
 (318, 1),
 (319, 2),
 (320, 1),
 (321, 2),
 (322, 1),
 (323, 1),
 (324, 1),
 (325, 1),
 (326, 1),
 (327, 14),
 (328

In [24]:
[(idx,word) for idx, word in word_dict.items()]
## 0번 단어는 가족...
## 1번 단어는 가지...

[(0, '가슴'),
 (1, '가지'),
 (2, '감동'),
 (3, '감사'),
 (4, '감정이입'),
 (5, '개인'),
 (6, '게시물'),
 (7, '겨울'),
 (8, '겨울왕국'),
 (9, '고전'),
 (10, '공간'),
 (11, '공식'),
 (12, '공주'),
 (13, '광산'),
 (14, '국민'),
 (15, '궁전'),
 (16, '근처'),
 (17, '기념'),
 (18, '기념품'),
 (19, '기억'),
 (20, '꼬마'),
 (21, '끝자락'),
 (22, '나라'),
 (23, '난쟁이'),
 (24, '남자'),
 (25, '노래'),
 (26, '누디드'),
 (27, '누디트'),
 (28, '느낌'),
 (29, '다람쥐'),
 (30, '덕후'),
 (31, '데이트'),
 (32, '도모'),
 (33, '도착'),
 (34, '독립'),
 (35, '동물'),
 (36, '동생'),
 (37, '동영상'),
 (38, '뒷배경'),
 (39, '등불'),
 (40, '디즈니'),
 (41, '뚝섬역'),
 (42, '라푼젤'),
 (43, '레거시'),
 (44, '마감'),
 (45, '마무리'),
 (46, '마법'),
 (47, '마부'),
 (48, '마음'),
 (49, '마지막'),
 (50, '마차'),
 (51, '만화'),
 (52, '머리'),
 (53, '명장면'),
 (54, '몽환'),
 (55, '문구'),
 (56, '문화'),
 (57, '미녀'),
 (58, '미키마우스'),
 (59, '바닥'),
 (60, '발자국'),
 (61, '발전'),
 (62, '배경'),
 (63, '백설'),
 (64, '백설공주'),
 (65, '변화'),
 (66, '별명'),
 (67, '보기'),
 (68, '보라색'),
 (69, '보석'),
 (70, '본격'),
 (71, '블로거'),
 (72, '비둘기'),
 (73, '사진'),
 (74, '새틴'),
 (75,

## LDA 모델 훈련 (Topic 수: 30개로 설정)

In [25]:
import gensim

In [26]:
N_TOPICS = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics = N_TOPICS,  ## 주제 몇 개 할 건지
                                           id2word=word_dict)   ## vocabulary set

### 각 topic의 대표 단어들을 출력

In [27]:
topics = ldamodel.print_topics(num_words=5)  ## 각 토픽들에 대해서 몇 개의 단어(num_words)로 표현할 것인지
## 대표적인 단어를 알 수 있음
for topic in topics:
    print(topic)

(6, '0.006*"북미" + 0.005*"디즈니" + 0.004*"애니메이션" + 0.004*"해외" + 0.004*"파묘"')
(16, '0.007*"애니메이션" + 0.004*"북미" + 0.004*"해외" + 0.003*"디즈니" + 0.003*"게임"')
(26, '0.005*"넷플릭스" + 0.003*"웡카" + 0.003*"애니메이션" + 0.003*"등급" + 0.002*"버스"')
(2, '0.005*"월드" + 0.005*"해외" + 0.004*"북미" + 0.004*"애니메이션" + 0.003*"디즈니"')
(14, '0.006*"애니메이션" + 0.005*"문단속" + 0.005*"스즈메의" + 0.005*"월드" + 0.004*"해외"')
(24, '0.005*"디즈니" + 0.005*"마블" + 0.003*"배트맨" + 0.003*"넷플릭스" + 0.003*"애니메이션"')
(23, '0.007*"디즈니" + 0.005*"넷플릭스" + 0.005*"아바타" + 0.003*"달러" + 0.003*"월드"')
(1, '0.010*"달러" + 0.007*"해외" + 0.005*"월드" + 0.005*"디즈니" + 0.005*"북미"')
(18, '0.004*"달러" + 0.003*"버스" + 0.003*"멀티" + 0.003*"스파이더맨" + 0.003*"북미"')
(4, '0.004*"디즈니" + 0.004*"판매" + 0.003*"좌석" + 0.003*"애니메이션" + 0.003*"코미디"')
(20, '0.006*"웡카" + 0.005*"애니메이션" + 0.004*"해외" + 0.004*"넷플릭스" + 0.003*"블랙"')
(27, '0.005*"넷플릭스" + 0.005*"애니메이션" + 0.005*"디즈니" + 0.004*"달러" + 0.004*"북미"')
(28, '0.003*"월드" + 0.003*"아바타" + 0.003*"극장판" + 0.003*"미국" + 0.002*"북미"')
(8, '0.007*"범죄도시" + 0.004

## LDA 시각화

In [28]:
from IPython.core.display import HTML
import pyLDAvis
from pyLDAvis import gensim_models

In [29]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, word_dict)
pyLDAvis.display(vis)

## 문서 별 Topic 분포

In [30]:
for i, topic_list in enumerate(ldamodel[corpus]):
    print(f'{i}번째 문서의 topic 비중: {topic_list}')
    if i==5:
        break

0번째 문서의 topic 비중: [(23, 0.038901124), (27, 0.9581567)]
1번째 문서의 topic 비중: [(20, 0.9946827)]
2번째 문서의 topic 비중: [(5, 0.44213104), (10, 0.4494317), (13, 0.027528422), (16, 0.019357054), (20, 0.0508797)]
3번째 문서의 topic 비중: [(5, 0.08698558), (7, 0.23652864), (12, 0.34678042), (21, 0.3257234)]
4번째 문서의 topic 비중: [(5, 0.24201307), (10, 0.013390054), (13, 0.20142071), (14, 0.54065704)]
5번째 문서의 topic 비중: [(15, 0.80170906), (29, 0.195365)]


### 각 문서에 대해서 Topic의 비중을 나타내는 함수 작성

In [31]:
## 교수님 코드 

def get_topic_ratio_for_each_document(ldamodel, corpus):
    results = []
    for i, topic_list in enumerate(ldamodel[corpus]):
        sorted_topic_list = sorted(topic_list, key = lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list]
        results.append(doc_result)
        
    results = pd.DataFrame(results, columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])
    return results

In [32]:
df_results = get_topic_ratio_for_each_document(ldamodel, corpus)

df_results

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,27,0.960225,"[(27, 0.96022475), (23, 0.036833115)]"
1,1,20,0.994683,"[(20, 0.9946827)]"
2,2,10,0.444744,"[(10, 0.44474405), (5, 0.43501467), (20, 0.052..."
3,3,12,0.344242,"[(12, 0.3442423), (21, 0.3288201), (7, 0.24037..."
4,4,14,0.541961,"[(14, 0.5419613), (5, 0.24252966), (13, 0.2003..."
...,...,...,...,...
1897,1897,2,0.994706,"[(2, 0.9947057)]"
1898,1898,0,0.892941,"[(0, 0.8929415), (10, 0.08837253)]"
1899,1899,23,0.942401,"[(23, 0.9424013), (5, 0.048601314)]"
1900,1900,4,0.997787,"[(4, 0.99778664)]"


In [33]:
## 참고 사이트: https://wikidocs.net/30708

def make_topictable_per_doc(ldamodel, corpus):
    # 데이터를 저장할 빈 리스트 초기화
    topic_data = []

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_data.append([i, int(topic_num), round(prop_topic, 4), topic_list])
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
            
    # 리스트를 데이터프레임으로 변환
    topic_table = pd.DataFrame(topic_data, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])

    return(topic_table)

In [34]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable[:10]

,index,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0,27,0.9633,"[(23, 0.033715695), (27, 0.9633421)]"
1,1,1,20,0.9947,"[(20, 0.9946827)]"
2,2,2,10,0.4436,"[(5, 0.43407348), (10, 0.44362387), (13, 0.012..."
3,3,3,12,0.3447,"[(5, 0.084653825), (7, 0.23891795), (12, 0.344..."
4,4,4,14,0.5454,"[(5, 0.23980358), (10, 0.010803621), (13, 0.20..."
5,5,5,15,0.8046,"[(15, 0.8046016), (29, 0.1924724)]"
6,6,6,0,0.9910,"[(0, 0.9910457)]"
7,7,7,20,0.5392,"[(0, 0.21305624), (5, 0.24567144), (20, 0.5392..."
8,8,8,20,0.6100,"[(4, 0.38280493), (20, 0.6099565)]"
9,9,9,8,0.9926,"[(8, 0.99258006)]"


In [35]:
## 교수님 코드 + 각 토픽 별 키워드를 볼 수 있는 함수 (GPT의 도움을 조금 받음!)


def get_topic_keywords(ldamodel, num_keywords=5):
    topic_keywords = {}
    for topic_id in range(ldamodel.num_topics):
        keywords = [word for word, _ in ldamodel.show_topic(topic_id, topn=num_keywords)]
        topic_keywords[topic_id] = keywords
    return topic_keywords

def get_topic_ratio_for_each_document(ldamodel, corpus):
    topic_keywords = get_topic_keywords(ldamodel)
    results = []
    for i, topic_list in enumerate(ldamodel[corpus]):
        sorted_topic_list = sorted(topic_list, key = lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        second_most_important_topic, second_most_important_ratio = sorted_topic_list[1] if len(sorted_topic_list) > 1 else (None, 0)

        keywords_1 = ", ".join(topic_keywords[most_important_topic])
        keywords_2 = ", ".join(topic_keywords[second_most_important_topic]) if second_most_important_topic is not None else ''
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list, keywords_1, keywords_2]
        results.append(doc_result)
        
    results = pd.DataFrame(results, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중', 'Top1_Topic_keyword', 'Top2_Topic_keyword'])
    return results

In [36]:
df_results = get_topic_ratio_for_each_document(ldamodel, corpus)

df_results

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중,Top1_Topic_keyword,Top2_Topic_keyword
0,0,27,0.959272,"[(27, 0.9592718), (23, 0.037786044)]","넷플릭스, 애니메이션, 디즈니, 달러, 북미","디즈니, 넷플릭스, 아바타, 달러, 월드"
1,1,20,0.994683,"[(20, 0.9946827)]","웡카, 애니메이션, 해외, 넷플릭스, 블랙",
2,2,10,0.445629,"[(10, 0.44562906), (5, 0.4382672), (20, 0.0544...","북미, 달러, 월드, 와이드, 해외","북미, 월드, 애니메이션, 해외, 토마토"
3,3,12,0.343261,"[(12, 0.3432608), (21, 0.32786834), (7, 0.2388...","네이버, 애니메이션, 돌비, 포스터, 코미디","기자, 시네마, 달러, 독립, 돌비"
4,4,14,0.545253,"[(14, 0.54525304), (5, 0.238751), (13, 0.19479...","애니메이션, 문단속, 스즈메의, 월드, 해외","북미, 월드, 애니메이션, 해외, 토마토"
...,...,...,...,...,...,...
1897,1897,2,0.994706,"[(2, 0.9947057)]","월드, 해외, 북미, 애니메이션, 디즈니",
1898,1898,0,0.884217,"[(0, 0.88421726), (10, 0.09709664)]","달러, 범죄도시, 넷플릭스, 블랙, 마동석","북미, 달러, 월드, 와이드, 해외"
1899,1899,23,0.932994,"[(23, 0.9329938), (5, 0.058008824)]","디즈니, 넷플릭스, 아바타, 달러, 월드","북미, 월드, 애니메이션, 해외, 토마토"
1900,1900,4,0.997787,"[(4, 0.99778664)]","디즈니, 판매, 좌석, 애니메이션, 코미디",
